In [33]:
import pandas as pd
import numpy as np
df = pd.read_csv('indeed_job_dataset_short.csv')

In [34]:
df.shape

(5715, 13)

In [35]:
df.head()

,Job_Title,Queried_Salary,Job_Type,Skill,No_of_Skills,Company,No_of_Reviews,No_of_Stars,Date_Since_Posted,Location,Company_Revenue,Company_Employees,Company_Industry
0,Data Scientist,<80000,data_scientist,"['SAP', 'SQL']",2,Express Scripts,3301.0,3.3,1.0,MO,More than $10B (USD),"10,000+",Health Care
1,Data Scientist,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']",5,Money Mart Financial Services,NaN,NaN,15.0,TX,NaN,NaN,NaN
2,Data Scientist,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...",9,comScore,62.0,3.5,1.0,OR,NaN,NaN,NaN
3,Graduate Studies Program - Data Scientist,<80000,data_scientist,['Certified Internal Auditor'],1,Central Intelligence Agency,158.0,4.3,30.0,DC,NaN,NaN,Government
4,Data Scientist I,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",7,Federal Reserve Bank of Dallas,495.0,4.1,30.0,TX,NaN,"Less than 10,000",Banks and Financial Services


In [46]:
df_dum = pd.get_dummies(df['Job_Title'])

In [47]:
df_dum.shape

(5715, 2314)

In [ ]:
df_dum.to_csv(index=False)